- from dataset, readme 
- 연습 내용 중 하나 제안만 선택해 분석
- 결측치나 이상치는 분석가 판단해 처리
- 공통 주제 제안 4

1. 동일한 시기에 동일한 차종(product)을 선택했다고 했을 때, 
2. 사람들이 금액을 얼만큼(amount)을 사용했는지 최대와 미니멈, 그리고 평균을 확인 할 수 있게, 
3. 또한 차종별 가장 많은 돈을 쓴 비율에 많은 세대와 반대로 적은 돈을 쓴 세대는 어디인지? 확인하기
4. 특정한 세대를 통해 다른 데이터들의 상관관계 분석후 +1에 가까운 컬럼을 통해 전략 세우기 

In [72]:
import matplotlib.pyplot as plt
import matplotlib as mpl # 한글 폰트 설정 (NanumGothic) 
mpl.rcParams['font.family'] = 'NanumGothic'
mpl.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
import pandas as pd 
rentAcar = pd.read_csv('../../datasets/LetalCarOfContractType.csv')
rentAcar.head()

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.0,2363.0
3,66423450,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,계약확정,0,없음,5.0,농협은행,정상,60.0,2449.0
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,12,있음,8.0,농협은행,해약,51.0,1942.0


In [73]:
# rentAcar['product'].value_counts()

In [74]:
K1Df = rentAcar[rentAcar['product'] == 'K1']
K2Df = rentAcar[rentAcar['product'] == 'K2']
K3Df = rentAcar[rentAcar['product'] == 'K3']
K4Df = rentAcar[rentAcar['product'] == 'K4']
K5Df = rentAcar[rentAcar['product'] == 'K5']
K6Df = rentAcar[rentAcar['product'] == 'K6']


In [75]:
# datetime 컬럼 변환
rentAcar['datetime'] = pd.to_datetime(rentAcar['datetime'])

# 연령대 함수 수정 (Unknown 포함)
def get_age_group(age):
    if pd.isna(age):
        return 'Unknown'
    return f"{(age//10)*10}"

# 연령대 컬럼 추가
rentAcar['age'] = rentAcar['age'].apply(get_age_group)

# 날짜별, 차종별, 연령대별 그룹화하여 통계 계산
grouped_stats = rentAcar.groupby(['datetime', 'product', 'age'])['amount'].agg([
    'min',
    'max',
    'mean',
    'count'
]).reset_index()

In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 연령대 함수 수정
def get_age_group(age):
    if pd.isna(age):
        return 'Unknown'
    # 숫자만 있는 경우
    age = int(float(age))
    return f"{(age//10)*10}대"

# 임시로 연령대 컬럼 생성
rentAcar['temp_age_group'] = rentAcar['age'].apply(get_age_group)

# 차종별 통계 계산 (Unknown 제외)
car_age_stats = (rentAcar[rentAcar['temp_age_group'] != 'Unknown']
                .groupby(['product', 'temp_age_group'])['amount']
                .agg(['count', 'min', 'max', 'mean'])
                .round(2))

# 각 차종별 분석
for car in sorted(rentAcar['product'].unique()):
    try:
        # 해당 차종의 데이터만 필터링
        car_subset = rentAcar[rentAcar['product'] == car]
        car_stats = car_age_stats.loc[car]
        
        # 최소 표본 수 기준 설정 (10개 이상의 데이터가 있는 연령대만 고려)
        valid_groups = car_stats[car_stats['count'] >= 10]
        
        if not valid_groups.empty:
            print(f"\n{car} 분석 결과:")
            print(f"총 데이터 수: {car_subset.shape[0]:,}개")
            print(f"유효 데이터 수: {valid_groups['count'].sum():,}개")
            
            # 최대 지출 연령대
            max_spending_age = valid_groups['mean'].idxmax()
            print(f"\n최대 평균 지출 연령대: {max_spending_age}")
            print(f"  - 평균: {valid_groups.loc[max_spending_age, 'mean']:,.0f}원")
            print(f"  - 최소: {valid_groups.loc[max_spending_age, 'min']:,.0f}원")
            print(f"  - 최대: {valid_groups.loc[max_spending_age, 'max']:,.0f}원")
            print(f"  - 건수: {valid_groups.loc[max_spending_age, 'count']:,}건")
            
            # 최소 지출 연령대
            min_spending_age = valid_groups['mean'].idxmin()
            print(f"\n최소 평균 지출 연령대: {min_spending_age}")
            print(f"  - 평균: {valid_groups.loc[min_spending_age, 'mean']:,.0f}원")
            print(f"  - 최소: {valid_groups.loc[min_spending_age, 'min']:,.0f}원")
            print(f"  - 최대: {valid_groups.loc[min_spending_age, 'max']:,.0f}원")
            print(f"  - 건수: {valid_groups.loc[min_spending_age, 'count']:,}건")
            
            # Unknown 비율 계산
            unknown_count = car_subset['temp_age_group'].eq('Unknown').sum()
            unknown_ratio = (unknown_count / car_subset.shape[0]) * 100
            print(f"\nUnknown 데이터: {unknown_count:,}개 ({unknown_ratio:.1f}%)")
            
        else:
            print(f"\n{car}: 충분한 데이터가 없습니다 (최소 10개 이상 필요)")
            
    except Exception as e:
        print(f"\n{car} 처리 중 오류 발생: {str(e)}")

# 시각화
plt.figure(figsize=(15, 8))
for car in sorted(rentAcar['product'].unique()):
    try:
        car_stats = car_age_stats.loc[car]
        valid_groups = car_stats[car_stats['count'] >= 10]
        
        if not valid_groups.empty:
            plt.plot(valid_groups.index, valid_groups['mean'], 
                    marker='o', label=f'{car} (n={car_stats["count"].sum():,})')
    except:
        continue

plt.title('차종별 연령대 평균 지출액')
plt.xlabel('연령대')
plt.ylabel('평균 지출액 (원)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 임시 컬럼 제거
rentAcar.drop('temp_age_group', axis=1, inplace=True)


ValueError: could not convert string to float: 'Unknown'

In [ ]:
# 연령대를 계산하는 함수
def get_age_group(age):
    return f"{(age // 10) * 10}대"

# 데이터프레임 리스트 생성
dfs = [K1Df, K2Df, K3Df, K4Df, K5Df, K6Df]
df_names = ['K1', 'K2', 'K3', 'K4', 'K5', 'K6']

for df, name in zip(dfs, df_names):
    print(f"\n{'='*50}")
    print(f"{name} 데이터 분석 결과")
    print('='*50)
    
    # amount 기준으로 상위 25%와 하위 25% 데이터 추출
    top_25_mask = df['amount'] >= df['amount'].quantile(0.75)
    bottom_25_mask = df['amount'] <= df['amount'].quantile(0.25)
    
    # 각 그룹의 데이터 추출
    top_25 = df[top_25_mask].copy()
    bottom_25 = df[bottom_25_mask].copy()
    
    # 연령대 컬럼 추가
    top_25['age_group'] = top_25['age'].apply(get_age_group)
    bottom_25['age_group'] = bottom_25['age'].apply(get_age_group)
    
    # 각 그룹의 연령대별 분포 계산
    top_25_dist = top_25['age_group'].value_counts().sort_index()
    bottom_25_dist = bottom_25['age_group'].value_counts().sort_index()
    
    # 비율 계산
    top_25_ratio = (top_25_dist / len(top_25) * 100).round(2)
    bottom_25_ratio = (bottom_25_dist / len(bottom_25) * 100).round(2)
    
    print("\n상위 25% amount 그룹의 연령대별 분포:")
    print("인원수:")
    print(top_25_dist)
    print("\n비율(%):")
    print(top_25_ratio)
    
    print("\n하위 25% amount 그룹의 연령대별 분포:")
    print("인원수:")
    print(bottom_25_dist)
    print("\n비율(%):")
    print(bottom_25_ratio)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 결과를 저장할 빈 데이터프레임 생성
results = []

# 연령대를 계산하는 함수 수정
def get_age_group(age):
    if pd.isna(age):
        return 'Unknown'
    return f"{(age//10)*10}대"  # 수정된 부분

# 데이터프레임 리스트 생성
dfs = [K1Df, K2Df, K3Df, K4Df, K5Df, K6Df]
df_names = ['K1', 'K2', 'K3', 'K4', 'K5', 'K6']

# 각 데이터프레임에 대한 분석
for df, name in zip(dfs, df_names):
    # amount 기준으로 상위 25%와 하위 25% 데이터 추출
    top_25_mask = df['amount'] >= df['amount'].quantile(0.75)
    bottom_25_mask = df['amount'] <= df['amount'].quantile(0.25)
    
    # 각 그룹의 데이터 추출
    top_25 = df[top_25_mask].copy()
    bottom_25 = df[bottom_25_mask].copy()
    
    # 연령대 컬럼 추가
    top_25['age_group'] = top_25['age'].apply(get_age_group)
    bottom_25['age_group'] = bottom_25['age'].apply(get_age_group)
    
    # 각 그룹의 연령대별 분포 계산
    top_25_dist = top_25['age_group'].value_counts()
    bottom_25_dist = bottom_25['age_group'].value_counts()
    
    # 비율 계산
    top_25_ratio = (top_25_dist / len(top_25) * 100).round(2)
    bottom_25_ratio = (bottom_25_dist / len(bottom_25) * 100).round(2)
    
    # 결과 저장
    for age_group in sorted(set(top_25['age_group'].unique()) | set(bottom_25['age_group'].unique())):
        results.append({
            'Dataset': name,
            'Age_Group': age_group,
            'Percentage': top_25_ratio.get(age_group, 0),
            'Group': 'Top 25%'
        })
        results.append({
            'Dataset': name,
            'Age_Group': age_group,
            'Percentage': bottom_25_ratio.get(age_group, 0),
            'Group': 'Bottom 25%'
        })

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 연령대 순서 정의 (Unknown은 마지막에 표시)
age_order = sorted([x for x in results_df['Age_Group'].unique() if x != 'Unknown']) + ['Unknown']

# 그래프 생성을 위한 피벗 테이블 생성
def create_plot(data, title):
    plt.figure(figsize=(15, 8))
    
    # 피벗 테이블 생성
    pivot_data = data.pivot(index='Dataset', columns='Age_Group', values='Percentage').fillna(0)
    
    # 컬럼 순서 재정렬
    pivot_data = pivot_data[age_order]
    
    # 누적 막대 그래프 생성
    ax = pivot_data.plot(kind='bar', stacked=True, figsize=(15, 8))
    
    plt.title(title, fontsize=14, pad=20)
    plt.xlabel('데이터셋', fontsize=12)
    plt.ylabel('비율 (%)', fontsize=12)
    plt.legend(title='연령대', bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # 각 막대의 총합이 100%가 되도록 조정
    for c in ax.containers:
        ax.bar_label(c, fmt='%.1f%%', label_type='center')
    
    plt.tight_layout()
    plt.show()

# 상위 25% 그래프 생성
create_plot(results_df[results_df['Group'] == 'Top 25%'], '연령대별 분포 비교 (상위 25%)')

# 하위 25% 그래프 생성
create_plot(results_df[results_df['Group'] == 'Bottom 25%'], '연령대별 분포 비교 (하위 25%)')


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 이전 결과 데이터프레임(results_df)을 활용하여 차이 계산
def calculate_difference():
    # 상위 25%와 하위 25% 데이터 분리
    top_25 = results_df[results_df['Group'] == 'Top 25%']
    bottom_25 = results_df[results_df['Group'] == 'Bottom 25%']
    
    # 피벗 테이블 생성
    top_pivot = top_25.pivot(index='Dataset', columns='Age_Group', values='Percentage').fillna(0)
    bottom_pivot = bottom_25.pivot(index='Dataset', columns='Age_Group', values='Percentage').fillna(0)
    
    # 차이 계산 (절댓값)
    difference = abs(top_pivot - bottom_pivot)
    
    return difference

# 차이 계산
diff_df = calculate_difference()

# 시각화
plt.figure(figsize=(15, 8))

# 각 데이터셋별로 연령대 차이를 막대 그래프로 표시
x = np.arange(len(diff_df.index))
width = 0.8 / len(diff_df.columns)  # 막대 너비

# 각 연령대별로 막대 그래프 생성
for i, age_group in enumerate(diff_df.columns):
    plt.bar(x + i * width, diff_df[age_group], 
            width, label=age_group, alpha=0.7)

# 그래프 꾸미기
plt.xlabel('데이터셋', fontsize=12)
plt.ylabel('상위 25% - 하위 25% 차이 (절댓값, %)', fontsize=12)
plt.title('연령대별 상위 25%와 하위 25% 간의 차이', fontsize=14, pad=20)
plt.xticks(x + width * len(diff_df.columns) / 2, diff_df.index)
plt.legend(title='연령대', bbox_to_anchor=(1.05, 1), loc='upper left')

# 값 레이블 표시
for i, age_group in enumerate(diff_df.columns):
    for j, value in enumerate(diff_df[age_group]):
        if value > 0:  # 0보다 큰 값만 표시
            plt.text(j + i * width, value, f'{value:.1f}%', 
                    ha='center', va='bottom')

plt.tight_layout()
plt.show()

# 수치로도 확인
print("\n연령대별 상위 25%와 하위 25% 간의 차이 (절댓값):")
print(diff_df.round(2))


**K1**
- 가장 큰 차이: Unknown(11.75%), 40대(4.30%)
- 30대(2.31%)와 50대(1.75%)에서도 비교적 큰 차이
- 80대(0.07%), 90대(0.05%), 100대(0.01%)는 매우 작은 차이

**K2**
- 전반적으로 차이가 작은 편
- 가장 큰 차이: Unknown(4.45%), 20대(1.83%)
- 30대(0.76%), 40대(0.71%), 60대(0.70%) 비슷한 수준
- 90대(0.01%)에서 가장 작은 차이

**K3**
- Unknown(12.07%)이 가장 큰 차이
- 젊은층에서 큰 차이: 30대(3.64%), 20대(3.37%)
- 50대(2.36%), 40대(2.02%)도 상당한 차이
- 100대(0.00%)에서 차이 없음

**K4**
- 중장년층에서 가장 큰 차이: 60대(5.80%), 50대(4.66%)
- 70대(3.88%)도 높은 차이
- Unknown은 4.13%
- 젊은층은 상대적으로 작은 차이: 20대(0.28%), 30대(0.27%)
- 100대(0.00%)에서 차이 없음

**K5**
- 전 연령대에서 가장 작은 차이를 보임
- 최대 차이는 30대(1.43%)
- Unknown(0.02%)도 매우 작음
- 100대(0.00%)에서 차이 없음

**K6**
- Unknown(9.05%)이 가장 큰 차이
- 40대(2.50%)에서 가장 큰 실제 연령대 차이
- 30대와 50대 모두 1.91%로 동일
- 90대와 100대(0.00%)에서 차이 없음